# Hybrid CNN pipeline

### 1. Load CATPCHA images and get the transformed (X,y) data

In [2]:
import torch
import numpy as np

from helper_functions.data_transformation import (
    get_transformed_data, CHARACTERS, 
)

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"

print("Preparing training data...")
X_train, y_train_onehot, X_features_train, feature_names, scaler = get_transformed_data(train_folder)
y_train = torch.tensor(np.argmax(y_train_onehot, axis=1), dtype=torch.long)

print("Preparing test data...")
X_test,y_test_onehot, X_features_test, _, _ = get_transformed_data(test_folder)
y_test = torch.tensor(np.argmax(y_test_onehot, axis=1), dtype=torch.long)

Preparing training data...


Preparing Data: 100%|██████████| 7437/7437 [00:27<00:00, 275.39it/s]


Preparing test data...


Preparing Data: 100%|██████████| 1894/1894 [00:07<00:00, 251.54it/s]


### 2. Instantiate Hybrid CNN model and train it

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from helper_functions.models.hybrid_cnn import HybridDataset, HybridCNN

# Create dataloaders
train_loader = DataLoader(
    HybridDataset(X_train, X_features_train, y_train), 
    batch_size=32, shuffle=True
)
test_loader = DataLoader(
    HybridDataset(X_test, X_features_test, y_test), 
    batch_size=32, shuffle=False
)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridCNN(num_classes=len(CHARACTERS), num_features=len(feature_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print("Training model...")
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for (inputs_img, inputs_features), labels in train_loader:
        inputs_img = inputs_img.to(device)
        inputs_features = inputs_features.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()
        outputs = model(inputs_img, inputs_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs_img.size(0)
    
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader.dataset):.4f}")

Training model...
Epoch 1 Loss: 1.5133
Epoch 2 Loss: 0.7912
Epoch 3 Loss: 0.6125
Epoch 4 Loss: 0.5055
Epoch 5 Loss: 0.4340
Epoch 6 Loss: 0.3780
Epoch 7 Loss: 0.3307
Epoch 8 Loss: 0.2861
Epoch 9 Loss: 0.2547
Epoch 10 Loss: 0.2279
Epoch 11 Loss: 0.1998
Epoch 12 Loss: 0.1824
Epoch 13 Loss: 0.1669
Epoch 14 Loss: 0.1450
Epoch 15 Loss: 0.1347
Epoch 16 Loss: 0.1252
Epoch 17 Loss: 0.1127
Epoch 18 Loss: 0.1094
Epoch 19 Loss: 0.1004
Epoch 20 Loss: 0.1007
Epoch 21 Loss: 0.0903
Epoch 22 Loss: 0.0833
Epoch 23 Loss: 0.0786
Epoch 24 Loss: 0.0835
Epoch 25 Loss: 0.0730
Epoch 26 Loss: 0.0707
Epoch 27 Loss: 0.0661
Epoch 28 Loss: 0.0671
Epoch 29 Loss: 0.0662
Epoch 30 Loss: 0.0648
Epoch 31 Loss: 0.0527
Epoch 32 Loss: 0.0554
Epoch 33 Loss: 0.0580
Epoch 34 Loss: 0.0578
Epoch 35 Loss: 0.0521
Epoch 36 Loss: 0.0495
Epoch 37 Loss: 0.0458
Epoch 38 Loss: 0.0531
Epoch 39 Loss: 0.0424
Epoch 40 Loss: 0.0509
Epoch 41 Loss: 0.0449
Epoch 42 Loss: 0.0462
Epoch 43 Loss: 0.0413
Epoch 44 Loss: 0.0443
Epoch 45 Loss: 0.0489
E

### 3. Evaluation

In [3]:
from helper_functions.models.hybrid_cnn import evaluate_captcha_level, evaluate_character_level

# Evaluation
print("\nCharacter-Level Evaluation:")
char_metrics = evaluate_character_level(model, test_loader, device)
for metric, value in char_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}")

print("\nCaptcha-Level Evaluation:")
captcha_acc = evaluate_captcha_level(model, test_folder, device, scaler, feature_names)
print(f"Accuracy: {captcha_acc:.4f}")


Character-Level Evaluation:
Accuracy: 0.8108
Precision: 0.8116
Recall: 0.8105
F1: 0.8091

Captcha-Level Evaluation:


Evaluating CAPTCHAs: 100%|██████████| 1894/1894 [00:35<00:00, 53.58it/s]

Accuracy: 0.3099
